In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('cleaned_jobpostings.csv')

In [29]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the pre-trained model
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Example job descriptions (replace with your data)
job_descriptions = data['Job Description'].tolist()

# Generate embeddings
embeddings = model.encode(str(job_descriptions), convert_to_tensor=False)

In [41]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [42]:
connections.list_connections()

[('default', <pymilvus.client.grpc_handler.GrpcHandler at 0x2a1dce4f0>)]

In [43]:
#Connects to a server:
connections.connect(alias="default")

In [10]:
#Creates a collection:
fields = [
    FieldSchema(name="job_ids", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=250),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=384)
]
schema = CollectionSchema(fields, "e APIs")


In [12]:
dem= Collection("job_description_emb", schema)

In [13]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
dem.create_index("embeddings", index)

Status(code=0, message=)

In [14]:
embeddings= np.load('150k_job_description_embeddings.npy')

In [15]:
insert_data = [list(data['Job Id'][:100]), embeddings[:100]]

In [16]:
dem.insert(insert_data)
dem.flush()

In [22]:
dem.load()

In [17]:
search_params = {"metric_type": "L2", "params": {"nprobe": 84}, "offset": 0}

In [40]:
embeddings[0]

array([-0.10449833, -0.35488445, -0.14203393, -0.13399124, -0.26506042,
       -0.09482317, -0.17296463, -0.00837596, -0.13152482,  0.05788972,
       -0.1613378 , -0.01558806, -0.2649365 , -0.21986121,  0.24535067,
       -0.20539638, -0.0609545 , -0.24829572, -0.01366124, -0.028388  ,
        0.03680178, -0.30063057, -0.32515058, -0.13637915, -0.07780047,
       -0.2829086 , -0.17462933, -0.202629  ,  0.4618324 , -0.06900965,
        0.35098222, -0.36415038,  0.05271176,  0.05246148,  0.04971326,
        0.21795684, -0.25004366,  0.03673599, -0.2340553 , -0.39514476,
        0.2252634 ,  0.09092484, -0.08315606,  0.12313836,  0.0259116 ,
        0.13117276, -0.18161279,  0.44074547, -0.32069108,  0.32602736,
        0.04421189, -0.39924574, -0.22692627, -0.05094362,  0.12408137,
       -0.0939585 , -0.26367667,  0.31760854,  0.04235152, -0.35100338,
        0.20948306, -0.4115215 , -0.44280237,  0.13394184, -0.11459681,
       -0.17938474,  0.08254892,  0.20766312, -0.00273878, -0.19

In [23]:
encodings_10 = list(embeddings[:10])
results = dem.search(
    data=encodings_10, 
    anns_field="embeddings", 
    param=search_params,
    limit=2, 
    expr=None,
)

In [365]:
data[(data['Job Id']=='5c90575d962b567019643d4a8cea127f') | (data['Job Id']=='f8e7c2fe7bfda8455ca2e18eb1beea5a')]

,Job Id,Job Title,SOC Code,Job Description,Company Name,Skills,Qualification,City,State,Zipcode,Job Opening Date,Job Closing Date,Status,Website Url,stopwords
5,f8e7c2fe7bfda8455ca2e18eb1beea5a,Newborn Hearing Screener,29-2092.00,overview hearing screening associates fullserv...,Oticon USA,"[Patient Care, vaccine, Nursing, Auditory]",[High School Diploma],Concord,Massachusetts,1742.0,2021-09-09,2021-09-30,CLOSED,http://www.Oticon.com,152
6,5c90575d962b567019643d4a8cea127f,Newborn Hearing Screener,29-2092.00,overview hearing screening associates fullserv...,Oticon USA,"[Patient Care, vaccine, Nursing, Auditory]",[High School Diploma],Cambridge,Massachusetts,2138.0,2021-08-12,2021-09-30,CLOSED,http://www.Oticon.com,152


In [25]:
distances = [i.distances for i in results]
index_id = [i.ids for i in results]
#result_json = {'distances': distances, 'index_id': image_ids}

In [30]:
distances, index_id

([[0.0, 22.768169403076172],
  [0.0, 18.090246200561523],
  [0.0, 5.463075160980225],
  [0.0, 5.463075160980225],
  [0.0, 18.18899917602539],
  [0.0, 0.0],
  [0.0, 0.0],
  [0.0, 12.763259887695312],
  [0.0, 15.83663558959961],
  [0.0, 9.751897811889648]],
 [['89c41c519c3c491929e3082f0ee1d557', '67d926b6acca560280508da871fe12a4'],
  ['ac0c91f394fa77a00ad72ee3440cb4b7', '8ff27ad818f08b8cf0cfde5615bf4b58'],
  ['4b5748411c4496f56ef33645a27840e0', '7a7dac1bc98365216833008c0fbd063d'],
  ['7a7dac1bc98365216833008c0fbd063d', '4b5748411c4496f56ef33645a27840e0'],
  ['304aff90fd39fead183f48206f4070c7', '9fa7e748e5b3eb5d9c265f58c65b66ad'],
  ['5c90575d962b567019643d4a8cea127f', 'f8e7c2fe7bfda8455ca2e18eb1beea5a'],
  ['5c90575d962b567019643d4a8cea127f', 'f8e7c2fe7bfda8455ca2e18eb1beea5a'],
  ['d6583b8a3a0d7e72e884df39eda98576', '6db3c587c44d2ea173b2083d73a830f'],
  ['5c89e0132ec782188f22757c68c315a1', '67d926b6acca560280508da871fe12a4'],
  ['54d9be0a01d5c48a47116f5d527154fe', '146c645423347f2035327

In [327]:
#data.Zipcode.nunique()
np.unique(data.State)

array(['Massachusetts'], dtype=object)

In [320]:
data.drop_duplicates(subset=['Job Title', 'SOC Code', 'Job Description', 'Company Name',
       'Skills', 'Qualification', 'City', 'State', 'Zipcode',
       'Job Opening Date', 'Job Closing Date', 'Status', 'Website Url'])

,Job Id,Job Title,SOC Code,Job Description,Company Name,Skills,Qualification,City,State,Zipcode,Job Opening Date,Job Closing Date,Status,Website Url,stopwords
0,89c41c519c3c491929e3082f0ee1d557,"Editor, Celebrations",27-3041.00,gannett co inc nyse gci subscriptionled digita...,Gannett,"[Local Media, Editing, Journalism]","[Bachelor of Journalism (B.J.), Master of Jour...",Boston,Massachusetts,2108.0,2021-09-23,2021-09-30,CLOSED,https://www.gannett.com,350
1,ac0c91f394fa77a00ad72ee3440cb4b7,Software Engineer II,51-8021.00,overview reporting director data integration s...,ERT,"[Java, Application Architecture, CI, Data Stru...",[Bachelor of Computer Science (B.C.S.)],Medford,Massachusetts,2155.0,2021-04-29,2021-09-30,CLOSED,http://www.ert.com,263
2,4b5748411c4496f56ef33645a27840e0,Principal Software Architect,17-1011.00,digital technology forever changed business tr...,"Sovos Compliance, LLC.","[Java, CSS, Government Compliance, Global Comp...",NaN,Wilmington,Massachusetts,1887.0,2021-01-29,2021-09-30,CLOSED,https://sovos.com,299
3,7a7dac1bc98365216833008c0fbd063d,Strategy Program Manager,27-2012.03,build future sovos digital technology forever ...,"Sovos Compliance, LLC.","[Government Compliance, Global Compliance, Com...",NaN,Wilmington,Massachusetts,1887.0,2021-08-27,2021-09-30,CLOSED,https://sovos.com,205
4,304aff90fd39fead183f48206f4070c7,Senior Contract Manager,11-9199.00,description labella associates established 197...,LaBella Associates,"[Legal, Disciplinary]","[Bachelor of Engineering (B.E./B.Eng.), Any Ba...",Boston,Massachusetts,2108.0,2021-03-11,2021-09-30,CLOSED,http://www.labellapc.com,289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187848,459849bc27bbcb163fb705af2f01e678,Research Assistant I Lab,25-1191.00,auto req id 54560br job code 403108 research a...,Harvard University,"[In Vivo, Molecular Biology, Cloning, Gel Elec...",NaN,Boston,Massachusetts,2108.0,2021-07-10,NaN,OPEN,https://harvard.edu,165
187849,217b96a9bc7f005fbb75ee4c1cc40932,Finance and Admin Coor,43-1011.00,auto req id 54485br job code 403040 coordinato...,Harvard University,"[Accounting, General Administration]",NaN,Cambridge,Massachusetts,2138.0,2021-06-25,NaN,OPEN,https://harvard.edu,257
187850,f8fd3beb5dc778800272cf58aa9e8bb4,Systems Engineer,15-1199.02,auto req id 54656br job code i0958p infrastruc...,Harvard University,"[HTTP, ITIL, Software Development, HTTPS, Ansi...",NaN,Cambridge,Massachusetts,2138.0,2021-07-03,NaN,OPEN,https://harvard.edu,308
187851,a1e3777919425e3b6c1a8217fbefa7af,Property Operations Assistant,13-1199.00,auto req id 54025br job code 402065 property o...,Harvard University,NaN,NaN,Cambridge,Massachusetts,2138.0,2021-05-15,NaN,OPEN,https://harvard.edu,196
